<a href="https://colab.research.google.com/github/ratmonger/SentimentAnalysis/blob/main/python/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import chi2
from google.colab import drive
import re
import threading
import multiprocessing

Mounting to get access to Google Drive csv files

In [2]:
drive.mount('/content/drive')
train_csv = pd.read_csv("drive/MyDrive/RandomForest/train.csv", na_values=['NaN', 'NotFound', 'nan'], keep_default_na=True)

# touch up the CSV
# datasrcutures
# statistics to choose when to stop growing chi square
# MME, bootstrap testing
#info gain
# tree depth, accuracy, visuals
#contintuous features, missing values
#random forest
#imbalanced data
# hyperparameters
# sample features, sample instances, vary depth, chi square, bin

Mounted at /content/drive


In [3]:
train_csv = pd.read_csv("drive/MyDrive/RandomForest/test.csv", na_values=['NaN', 'NotFound', 'nan'], keep_default_na=True)
# train.head()
# shuffle data
impute_data(train_csv, attributes, method="median")

train_csv_shuffle = train_csv.sample(frac=1, random_state=10).reset_index(drop=True)
# split training data csv into 20% validation and 80% training. random_state set to arbitrary value -- for reproducing results
training_data, validation_data = train_test_split(train_csv_shuffle, test_size = 0.2, random_state = 10)
# training_data.head()
#validation_data.head()
print(train_csv)
training_data = train_csv.loc[( train_csv['isFraud'] == 1)   ]
nan_counts = training_data.isnull().sum()
# print(nan_counts)

train_csv = train_csv.loc[( train_csv['isFraud' == 1])   ]

for identifier, nan in nan_counts.items():
  print(identifier, nan / len(training_data))

feature_type = {
    'ProductCD': 'categorical',
    'card1': 'categorical',
    'card2': 'categorical',
    'card3': 'categorical',
    'card4': 'categorical',
    'card5': 'categorical',
    'card6': 'categorical',
    'addr1': 'categorical',
    'addr2': 'categorical',
    'TransactionDT': 'numerical',
    'TransactionAmt': 'numerical',
    'C1': 'numerical',
    'C2': 'numerical',
    'C3': 'numerical',
    'C4': 'numerical',
    'C5': 'numerical',
    'C6': 'numerical',
    'C7': 'numerical',
    'C8': 'numerical',
    'C9': 'numerical',
    'C10': 'numerical',
    'C11': 'numerical',
    'C12': 'numerical',
    'C13': 'numerical',
    'C14': 'numerical'
}

df = train_csv.dropna()
df['isFraud'].value_counts()

NameError: name 'impute_data' is not defined

In [ ]:
confidence = .95
df = 2

critical_chi2_value = chi2.ppf(confidence, df)
print(critical_chi2_value)



5.991464547107979


Class containing Tree information

In [4]:
class TreeNode: # tree node class
  def __init__(self, children={}, attribute=None, node_type=None, class_value=None, split=None, data = None, parent = None, parent_branch = None, chi_val=None,c_val = None,info_gain = None):
    self.children = children # dictionary of {keys:attribute value branches to values:child nodes}
    self.attribute = attribute # best attribute chosen to split at node
    self.node_type = node_type # type of node (leaf or decision node)
    self.class_value = class_value # target class value if leaf
    self.split = split # if continuous feature, value split at
    self.data = data # store instances associated with leaf node if leaf node
    self.parent = parent
    self.parent_branch = parent_branch
    self.chi_val = chi_val
    self.c_val = c_val
    self.info_gain = info_gain

  def __repr__(self):
        return f'TreeNode(parent={self.parent}, parent_branch={self.parent_branch}, attribute={self.attribute}, node_type={self.node_type}, class_value={self.class_value}, split={self.split}, c_val={self.c_val}, chi_val={self.chi_val}, max_info_gain={self.info_gain})'

Class containing Decision Tree information

In [5]:
class DecisionTree:
  def __init__(self, dataset, target_class, attributes, id='TransactionID', max_depth = None, impurity_measure = "entropy", confidence = 0.95, method="median"):
    self.dataset = dataset
    self.target_class = target_class # attribute to be predicted by the tree
    self.attributes = attributes # dictionary of features in training data and corresponding feature type (continuous or categorical)
    self.impurity_measure = impurity_measure
    self.max_depth = max_depth # hyperparameter
    self.confidence = confidence
    self.method = method # for imputation; either mean or median for numerical values
    self.tree = None # decision tree to be built
    self.id = id

  # method to calculate impurity with gini index for a given attribute
  # ranges from 0 to (1 - 1/n)
  def calculate_gini(self, data):
    if isinstance(data[self.target_class], pd.Series):
      if data.empty:
        return 1
      proportion = data[self.target_class].value_counts() / len(data)

      gini = 1 - np.sum(proportion ** 2)

      return gini

  # method to calculate impurity with entropy for a given attribute
  # ranges from 0 to log2(n) where n is the number of values in the attribute
  def calculate_entropy(self, data):
    if isinstance(data[self.target_class], pd.Series):
      if data.empty:
        return 0
      #print(data[self.target_class].value_counts())
      proportion = data[self.target_class].value_counts() / len(data)

      proportion = proportion[proportion != 0]
      entropy = np.sum(-(proportion * np.log2(proportion)))

      return entropy

  # method to calculate impurity with misclassification error for a given attribute
  def calculate_misclassification_error(self, data):
    if isinstance(data[self.target_class], pd.Series):
      proportion = data[self.target_class].value_counts() / len(data)
      if proportion.empty:
        return 1
      misclassification_error = 1 - max(proportion)

      return misclassification_error

  # calculate impurity based on selected impurity measure
  def calculate_impurity(self, data):
    if self.impurity_measure == "gini":
      return self.calculate_gini(data)
    elif self.impurity_measure == "entropy":
      return self.calculate_entropy(data)
    elif self.impurity_measure == "misclassification_error":
      return self.calculate_misclassification_error(data)

  def calculate_information_gain(self, data, attribute, split=None):
    impurity_parent = self.calculate_impurity(data)
    impurity_children_avg = 0.0


    # calculate information gain for continuous features
    if split is not None:
        split_data_left = data[data[attribute] <= split]
        split_data_right = data[data[attribute] > split]
        impurity_children_avg += (len(split_data_left) / len(data)) * self.calculate_impurity(split_data_left)
        impurity_children_avg += (len(split_data_right) / len(data)) * self.calculate_impurity(split_data_right)
        information_gain = impurity_parent - impurity_children_avg
        return information_gain, split

    # group data by the attribute and calculate impurity for each group
    grouped_data = data.groupby(attribute, group_keys=True)
    imp_groups = grouped_data.apply(lambda group: self.calculate_impurity(group))
    value_counts = grouped_data.size() / len(data)

    impurity_children_avg = np.sum(value_counts * imp_groups)

    information_gain = impurity_parent - impurity_children_avg
    return information_gain, None



  def find_best_split(self, data, attribute):
    values = data[attribute].unique()
    if len(values) <= 1:
        return None

    values = pd.to_numeric(values, errors='coerce')
    values.sort()

    values = np.array(values)  # Convert to numpy array for vectorized operations

    midpoints = np.quantile(values,[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1])  # calculate midpoint between consecutive values

    # Check if target value differs, calculate information gain based on split using midpoint value
    temp_info_gains = np.array([self.calculate_information_gain(data, attribute, split) for split in midpoints])

    best_split_index = np.argmax(temp_info_gains[:, 0])
    best_split = midpoints[best_split_index]

    return best_split



  def chi_square_test(self, data, attribute, split=None):
     # class distribution of the data
    data_counts = data[self.target_class].value_counts()
    # size of the data
    data_length = len(data)
    c_value = 0.0

    if split is not None:
      # list of possible values of attribute
      classes = data[self.target_class].unique()


      # alpha = 1 - self.confidence
      degrees_freedom = (len(self.dataset[self.target_class].unique()) - 1)
      chi_square_value = chi2.ppf(self.confidence, degrees_freedom)

      # iterate over the possible branches

      split_data_left = data[data[attribute] <= split]

      left_length = len(split_data_left)
      target_counts = split_data_left[self.target_class].value_counts()

          # create arrays for calculations
      expectations = left_length * (data_counts / data_length).values

      observed_counts = np.array([target_counts.get(identifier, 0) for identifier in data_counts.index])


          # vectorized calculation of c_value
      if len(expectations[expectations != 0]) > 0:
        c_value += np.sum(((observed_counts - expectations) ** 2) / expectations[expectations != 0])

      split_data_right = data[data[attribute] > split]

      right_length = len(split_data_right)
      target_counts = split_data_right[self.target_class].value_counts()

          # create arrays for calculations
      expectations = right_length * (data_counts / data_length).values
      observed_counts = np.array([target_counts.get(identifier, 0) for identifier in data_counts.index])

          # vectorized calculation of c_value
      if len(expectations[expectations != 0]) > 0:
        c_value += np.sum(((observed_counts - expectations) ** 2) / expectations[expectations != 0])

    if split is None:
      # list of possible values of attribute
      values = data[attribute].unique()
      classes = data[self.target_class].unique()

      # class distribution of the data


      # size of the data
      data_length = len(data)
      # alpha = 1 - self.confidence
      degrees_freedom = (len(self.dataset[attribute].unique())-1)*(len(self.dataset[self.target_class].unique()) - 1)
      chi_square_value = chi2.ppf(self.confidence, degrees_freedom)

      # iterate over the possible branches
      for value in values:
        data_subset = data[data[attribute] == value]
        subset_length = len(data_subset)
        target_counts = data_subset[self.target_class].value_counts()

          # create arrays for calculations
        expectations = subset_length * (data_counts / data_length).values
        observed_counts = np.array([target_counts.get(identifier, 0) for identifier in data_counts.index])

          # vectorized calculation of c_value
        c_value += np.sum(((observed_counts - expectations) ** 2) / expectations[expectations != 0])


    # print('CVAL:', c_value)
    # print('CHI', chi_square_value)

    return (chi_square_value > c_value), c_value, chi_square_value




  # method for imputation of missing data (mean, median, mode)
  def impute_data(self, data, method="median"):
    for column in data.columns[1:-1]:
      if (self.attributes[column] == 'categorical'):
        data[column].fillna(data[column].mode().iloc[0], inplace = True)
      else:
        if method == "mean":
          data[column] = pd.to_numeric(data[column], errors='coerce')
          data[column].fillna(data[column].mean(), inplace = True)
        elif method == "median":
          data[column] = pd.to_numeric(data[column], errors='coerce')
          data[column].fillna(data[column].median(), inplace = True)

  # recursively build the decision tree! id3 algorithm
  def build_tree(self, data, remaining_attributes, current_depth = 0, parent=None, parent_branch=None):

    #print(f"1: Best Attribute: n/a, Best Split: n/a")
    # if all instances are of the same class, create a leaf node
    if len(data[self.target_class].unique()) == 1 or current_depth >= self.max_depth:
        #print("mode supposedly", data[self.target_class].value_counts().iloc[0])
        leaf_node = TreeNode(node_type="leaf", class_value= data[self.target_class].mode().iloc[0], data = data, parent=parent, parent_branch = parent_branch,c_val=None,chi_val=None)
        #print(leaf_node.data)
        return leaf_node

    best_attribute = None
    max_information_gain = -1
    best_split = None # for continiuous features



    information_gains = pd.Series(remaining_attributes.keys()).apply(lambda attribute: self.calculate_information_gain(data, attribute)
                                                                     if remaining_attributes[attribute] != "numerical"
                                                                     else self.calculate_information_gain(data, attribute, self.find_best_split(data, attribute)))

    information_gains.index = remaining_attributes.keys()

    #information_gains_df = information_gains.to_frame(name='Information Gain')

    best_attribute = information_gains.apply(lambda x: x[0]).idxmax()

    max_info_gain = information_gains.apply(lambda x: x[0]).max()

    # Get the corresponding identifier
    best_split = information_gains[best_attribute][1]


    # Convert series of tuples to DataFrame with two columns
    #information_gains_df = information_gains.apply(lambda x: pd.Series({'Column1': x[0], 'Column2': x[1]}))

    #print("info gains 11111111111111111")
    #print(information_gains_df)



    #print(f"2: Best Attribute: {best_attribute}, Best Split: {best_split}")
    # if no best attribute found create leaf node
    if best_attribute is None:
      leaf_node = TreeNode(node_type="leaf", class_value=data[self.target_class].mode().iloc[0], data = data, parent=parent, parent_branch = parent_branch,c_val=-1,chi_val=-1)
      #print("data", leaf_node.data)
      return leaf_node

    #print(f"3: Best Attribute: {best_attribute}, Best Split: {best_split}")
    # do chi square test; if chi value is greater than return a leaf node
    chi_test, c_val, chi_val = self.chi_square_test(data, best_attribute, best_split)

    if chi_test:
      leaf_node = TreeNode(node_type="leaf", class_value=data[self.target_class].mode().iloc[0], data = data, parent=parent, parent_branch = parent_branch,c_val=c_val,chi_val=chi_val)
      return leaf_node

    subsets = {} # data to be passed on to each child based on value or split
    child_nodes ={}

    # intstead of deleting from attributes list create a current attributes list for a branch? idk
    new_attributes_list = remaining_attributes.copy()
    if new_attributes_list[best_attribute] == "categorical":
      new_attributes_list.pop(best_attribute)

    # split data
    if self.attributes[best_attribute] == "numerical":
      subsets["left"] = data[data[best_attribute] <= best_split]
      subsets["right"] = data[data[best_attribute] > best_split]
    else:
      # want to split based on values (not necessarily a binary split)
      for value in data[best_attribute].unique():
        single_subset = data[data[best_attribute] == value] # get subset based on value
        #print(single_subset)
        subsets[value] = single_subset


    #print(f"4: Best Attribute: {best_attribute}, Best Split: {best_split}")


    for subset in subsets:
      #print("testing subset print here!!!")
      #print(subsets[subset])
      child_nodes[subset] = self.build_tree(subsets[subset], new_attributes_list, current_depth +1, parent=best_attribute, parent_branch = subset)
    #print("child nodes print here")
    #print(child_nodes)

    # recursively build tree
    node = TreeNode(children = child_nodes, attribute = best_attribute, split = best_split, node_type = "decision", data = data, parent=parent, parent_branch = parent_branch,c_val=c_val,chi_val=chi_val, info_gain = max_info_gain)
    #recurse on subset data

    return node

  # method to train data
  def train(self):
    self.impute_data(self.dataset)
    print(self.dataset)
    self.tree = self.build_tree(self.dataset, self.attributes)

  # essentially want to traverse the tree and try to categorize the instance
  def predict_instance(self, instance, node):
    # return the class value if reach a leaf node
    if node.node_type == "leaf":
      #print( {instance['Day']: root_node.class_value})
      #print("predict instance", {instance["Day"]: instance[node.attribute]})
      return {self.id: instance[self.id], "Class": node.class_value}
    else:
      # get the value associated with the instance's given attribute (compared to current node's attribute)
      value = instance[node.attribute]
      #print(instance['Day'] )

      # traverse the tree
      # check if the node's attribute is numerical or categorical; if numerical, check split
      if self.attributes[node.attribute] == "numerical":
        if value <= node.split:
          # search nodes on left subtree if the attribute value of the instance is less than or equal to the split
          return self.predict_instance(instance, node.children["left"])
        else:
          # search nodes on the right subtree if the attribute value of the instance is greater than the split
          return self.predict_instance(instance, node.children["right"])
      # categorical feature
      else:
        if value in node.children:
        # traverse the branch that contains the value associatd with the instance's attribute
          return self.predict_instance(instance, node.children[value])
        else:
          return {self.id: instance[self.id], "Class": 0}

  def predict(self, test_data):
    '''predictions = [] # list of predicted values
    for i in range(len(test_data)):
      #print("test12333", test_data.iloc()[i])
      predictions.append(self.predict_instance(test_data.iloc()[i], self.tree))'''
    # .apply - pandas function to apply a function to a dataframe (df is test_data)
    # (lambda instance:) - lambda function that takes a single argument (instance) which represents a single row of the dataframe
    # lambda function calls the predict_instance function, applying the predict_instance function for every row of data in the dataframe
    # axis = 1: apply the function to rows
    predictions = test_data.apply(lambda instance: self.predict_instance(instance, self.tree), axis=1)
    # predictions is currently a Pandas series ; convert it to a list (maybe just sort the series later on?)
    predictions.tolist()

    #predictions_sorted = sorted(predictions, key=lambda x: extract_numeric_part(x["TransactionID"]))
    '''for prediction in predictions_sorted:
      print("prediction", prediction)'''
    #[print("prediction", prediction) for prediction in predictions]

    return predictions



Calculate accuracy of results

In [28]:
def impute_data(data, attributes, method="median"):
  for column in data.columns[1:-1]:
    if column == 'addr1':
      data[column].fillna(-1, inplace = True)
    elif (attributes[column] == 'categorical'):
      data[column].fillna(data[column].mode().iloc[0], inplace = True)
    else:
      if method == "mean":
        data[column] = pd.to_numeric(data[column], errors='coerce')
        data[column].fillna(data[column].mean(), inplace = True)
      elif method == "median":
        data[column] = pd.to_numeric(data[column], errors='coerce')
        data[column].fillna(data[column].median(), inplace = True)

In [ ]:
def calculate_accuracy(actual_values, predictions):
  print(actual_values)
  print(predictions)
  return (sum(actual == predict for actual, predict in zip(actual_values, predictions))) / len(actual_values)

ChatGPT code that utilizes regex to sort dictionary values. Just want to sort Days correctly

In [ ]:
def extract_numeric_part(day_value):
    match = re.match(r'\D*(\d+)', day_value)
    return int(match.group(1)) if match else 0

Do a Depth-First Search on the Tree to print out nodes

In [ ]:
def dfs(node):
    if node is not None:
        # print data of the current node
        print(repr(node))

        for child in node.children.values():
          dfs(child)

In [ ]:
def under_sample(data, proportion=0.06):
# Find the indices of instances with class_to_remove
  indices_to_remove = data.index[data['isFraud'] == 0]

  indices_minority = data.index[data['isFraud'] == 1]

# Choose a random subset of indices to keep (adjust the proportion as needed)

  indices_to_keep = np.random.choice(indices_to_remove, size=int(len(indices_to_remove) * proportion), replace=False)

# Create a mask for instances to keep
  union_indices = np.union1d(indices_to_keep, indices_minority)
  mask_to_keep = data.index.isin(union_indices)

# Apply the mask to the DataFrame
  return data[mask_to_keep]


In [ ]:
mitchell_csv = pd.read_csv("drive/MyDrive/RandomForest/mitchell.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
data = mitchell_csv.sample(frac=1, random_state=10).reset_index(drop=True)


print(data[data['Outlook'] == 'Sunny'])

data_types = {
    'Outlook': 'categorical',
    'Temperature': 'categorical',
    'Humidity': 'categorical',
    'Wind': 'categorical'
}
test_tree1 = DecisionTree(data, "PlayTennis", data_types, max_depth = 5, impurity_measure="entropy",confidence = 0.80, id='Day')

impurity_parent = test_tree1.calculate_impurity(data)
impurity_children_avg = 0.0

proportion = data["PlayTennis"].value_counts() / len(data)

print(proportion)

proportion = proportion[proportion != 0]
entropy = np.sum(-(proportion * np.log2(proportion)))
print(entropy)

# group data by the attribute and calculate impurity for each group
grouped_data = data.groupby("Outlook", group_keys=True)
#print(grouped_data)
value_counts = grouped_data.size() / len(data)
#print(value_counts.index)
#type(value_counts)
outlook_values = value_counts.index.tolist()
print(outlook_values)

imp_groups = grouped_data.apply(lambda group: test_tree1.calculate_impurity(group))
print("imp groups here")
print(imp_groups)
#impurity_children_avg = np.sum(value_counts * test_tree.calculate_impurity(grouped_data.get_group(outlook_value)) for outlook_value in outlook_values)
#information_gain = impurity_parent - impurity_children_avg

#print(information_gain)
impurity_children_avg = np.sum(value_counts * imp_groups)

information_gain = impurity_parent - impurity_children_avg

print(information_gain)



    Day Outlook Temperature Humidity    Wind PlayTennis
1    D8   Sunny        Mild     High    Weak         No
4    D9   Sunny        Cool   Normal    Weak        Yes
6   D11   Sunny        Mild   Normal  Strong        Yes
10   D2   Sunny         Hot     High  Strong         No
11   D1   Sunny         Hot     High    Weak         No
Yes    0.642857
No     0.357143
Name: PlayTennis, dtype: float64
0.9402859586706311
['Overcast', 'Rain', 'Sunny']
imp groups here
Outlook
Overcast    0.000000
Rain        0.970951
Sunny       0.970951
dtype: float64
0.24674981977443933


In [44]:
feature_type = {
    'ProductCD': 'categorical',
    'card1': 'categorical',
    'card2': 'categorical',
    'card3': 'categorical',
    'card4': 'categorical',
    'card5': 'categorical',
    'card6': 'categorical',
    'addr1': 'categorical',
    'addr2': 'categorical',
    'TransactionDT': 'numerical',
    'TransactionAmt': 'numerical',
    'C1': 'numerical',
    'C2': 'numerical',
    'C3': 'numerical',
    'C4': 'numerical',
    'C5': 'numerical',
    'C6': 'numerical',
    'C7': 'numerical',
    'C8': 'numerical',
    'C9': 'numerical',
    'C10': 'numerical',
    'C11': 'numerical',
    'C12': 'numerical',
    'C13': 'numerical',
    'C14': 'numerical'
}


train_csv = pd.read_csv("drive/MyDrive/RandomForest/train.csv", na_values=['NaN', 'NotFound', 'nan'], keep_default_na=True)
# train.head()
# shuffle data
#impute_data(train_csv, feature_type, method="median")

train_csv = train_csv.loc[( train_csv['isFraud'] == 1) ]
train_csv = train_csv.loc[ train_csv['card2'].isna()]
#print(train_csv)
print(np.sort(train_csv['addr1'].unique()))


train_csv_shuffle = train_csv.sample(frac=1, random_state=10).reset_index(drop=True)
# split training data csv into 20% validation and 80% training. random_state set to arbitrary value -- for reproducing results
training_data, validation_data = train_test_split(train_csv_shuffle, test_size = 0.2, random_state = 10)
# training_data.head()
#validation_data.head()
print(train_csv)
#training_data = train_csv.loc[( train_csv['isFraud'] == 1)   ]
nan_counts = training_data.isnull().sum()
# print(nan_counts)

#train_csv = train_csv.loc[( train_csv['isFraud' == 1])   ]

for identifier, nan in nan_counts.items():
  print(identifier, nan / len(training_data))



#df = train_csv.dropna()
#df['isFraud'].value_counts()

[123. 126. 143. 177. 181. 184. 191. 204. 205. 220. 225. 231. 264. 269.
 272. 299. 310. 324. 325. 327. 330. 337. 387. 469. 476. 498. 512.  nan]
        TransactionID ProductCD  card1  card2  card3 card4  card5   card6  \
842               843         W  15063    NaN    NaN   NaN    NaN     NaN   
4871             4872         C  15063    NaN  150.0  visa  226.0  credit   
5467             5468         W  15063    NaN  150.0  visa  226.0  credit   
5596             5597         C  15063    NaN  150.0  visa  226.0  credit   
6457             6458         C  15063    NaN  150.0  visa  226.0  credit   
...               ...       ...    ...    ...    ...   ...    ...     ...   
465676         465677         C  13832    NaN    NaN   NaN    NaN     NaN   
466706         466707         C  15063    NaN  150.0  visa  226.0  credit   
468685         468686         C  15063    NaN  150.0  visa  226.0  credit   
469475         469476         S   8187    NaN  150.0  visa  166.0   debit   
470375    

In [ ]:
iris_csv = pd.read_csv("drive/MyDrive/RandomForest/iris.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
iris_data = iris_csv.sample(frac=1, random_state=10).reset_index(drop=True) # shuffles data

iris_data.insert(0, 'identifier', range(1, 1 + len(iris_data)))


iris_types = {
    'sepal_length': 'numerical',
    'sepal_width': 'numerical',
    'petal_length': 'numerical',
    'petal_width': 'numerical'
}

# split training data csv into 20% validation and 80% training. random_state set to arbitrary value -- for reproducing results
training_data, validation_data = train_test_split(iris_data, test_size = 0.10, random_state = 10)

tree = DecisionTree(iris_data, "species", iris_types, max_depth = 1000, impurity_measure="gini", confidence = 0.51, id='identifier')
tree.train()

dfs(tree.tree)


print("=====================")
#dfs(test_tree.tree)
#print(mitchell_data)
predictions = tree.predict(validation_data)
#print(mitchell_data["PlayTennis"])

predictions_results = [dictionary["Class"] for dictionary in predictions if "Class" in dictionary]
print(calculate_accuracy(validation_data["species"], predictions_results) * 100)


#predictions = tree.predict(validation_data)
#print(mitchell_data["PlayTennis"])

#predictions_results = [dictionary["Class"] for dictionary in predictions if "Class" in dictionary]
#print(calculate_accuracy(validation_data["isFraud"], predictions_results) * 100)

     identifier  sepal_length  sepal_width  petal_length  petal_width  \
0             1           6.3          2.3           4.4          1.3   
1             2           6.4          2.7           5.3          1.9   
2             3           5.4          3.7           1.5          0.2   
3             4           6.1          3.0           4.6          1.4   
4             5           5.0          3.3           1.4          0.2   
..          ...           ...          ...           ...          ...   
145         146           5.7          2.5           5.0          2.0   
146         147           5.6          2.9           3.6          1.3   
147         148           5.7          4.4           1.5          0.4   
148         149           7.2          3.2           6.0          1.8   
149         150           4.9          3.1           1.5          0.1   

        species  
0    versicolor  
1     virginica  
2        setosa  
3    versicolor  
4        setosa  
..          ...

In [ ]:
train_csv = pd.read_csv("drive/MyDrive/RandomForest/train.csv", na_values=['NaN', 'NotFound', 'nan'], keep_default_na=True)
# train.head()
# shuffle data
train_csv = under_sample(train_csv)
train_csv_shuffle = train_csv.sample(frac=1, random_state=10).reset_index(drop=True)
# split training data csv into 20% validation and 80% training. random_state set to arbitrary value -- for reproducing results
training_data, validation_data = train_test_split(train_csv_shuffle, test_size = 0.2, random_state = 10, stratify=train_csv_shuffle['isFraud'])
print(training_data['isFraud'].value_counts())
print(validation_data['isFraud'].value_counts())
# training_data.head()
#validation_data.head()
nan_counts = training_data.isnull().sum()
# print(nan_counts)

#for identifier, nan in nan_counts.items():
#  print(identifier, nan / len(training_data))

feature_type = {
    'ProductCD': 'categorical',
    'card1': 'categorical',
    'card2': 'categorical',
    'card3': 'categorical',
    'card4': 'categorical',
    'card5': 'categorical',
    'card6': 'categorical',
    'addr1': 'categorical',
    'addr2': 'categorical',
    'TransactionDT': 'numerical',
    'TransactionAmt': 'numerical',
    'C1': 'numerical',
    'C2': 'numerical',
    'C3': 'numerical',
    'C4': 'numerical',
    'C5': 'numerical',
    'C6': 'numerical',
    'C7': 'numerical',
    'C8': 'numerical',
    'C9': 'numerical',
    'C10': 'numerical',
    'C11': 'numerical',
    'C12': 'numerical',
    'C13': 'numerical',
    'C14': 'numerical'
}


tree = DecisionTree(training_data, "isFraud", feature_type, max_depth = 5, impurity_measure="entropy", confidence = 0.95, id='TransactionID')
tree.train()

#dfs(tree.tree)

#predictions = tree.predict(validation_data)
#print(mitchell_data["PlayTennis"])

#predictions_results = [dictionary["Class"] for dictionary in predictions if "Class" in dictionary]
#print(calculate_accuracy(validation_data["isFraud"], predictions_results) * 100)

0    21883
1    13224
Name: isFraud, dtype: int64
0    5471
1    3306
Name: isFraud, dtype: int64
       TransactionID ProductCD  card1  card2  card3             card4  card5  \
6004          101709         W  14649  548.0  150.0              visa  226.0   
7155          131012         W   5376  111.0  150.0        mastercard  224.0   
10591          96324         W   6328  455.0  150.0        mastercard  166.0   
6204          326553         C  18084  111.0  150.0              visa  195.0   
18746           5055         H   9485  111.0  150.0              visa  226.0   
...              ...       ...    ...    ...    ...               ...    ...   
32452         274490         W   7585  553.0  150.0              visa  226.0   
32243         353524         R  17826  520.0  150.0  american express  190.0   
1603           49169         C   4504  500.0  185.0        mastercard  219.0   
6582           33596         H   9500  321.0  150.0              visa  226.0   
4815          303418  

0    364709
1     13236
Name: isFraud, dtype: int64

In [ ]:
impute_data(validation_data, feature_type, method="median")

predictions = tree.predict(validation_data)


predictions_results = [dictionary["Class"] for dictionary in predictions if "Class" in dictionary]
print(calculate_accuracy(validation_data["isFraud"], predictions_results) * 100)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Value:4806 Children:{14649: TreeNode(parent=card1, parent_branch=14649, attribute=C6, node_type=decision, class_value=None, split=1.3, c_val=52.03268717699957, chi_val=3.841458820694124, max_info_gain=0.4011647793278923), 5376: TreeNode(parent=card1, parent_branch=5376, attribute=None, node_type=leaf, class_value=1, split=None, c_val=None, chi_val=None, max_info_gain=None), 6328: TreeNode(parent=card1, parent_branch=6328, attribute=None, node_type=leaf, class_value=0, split=None, c_val=None, chi_val=None, max_info_gain=None), 18084: TreeNode(parent=card1, parent_branch=18084, attribute=None, node_type=leaf, class_value=0, split=None, c_val=4.0, chi_val=5.991464547107979, max_info_gain=None), 9485: TreeNode(parent=card1, parent_branch=9485, attribute=ProductCD, node_type=decision, class_value=None, split=None, c_val=27.42857142857143, chi_val=7.814727903251179, max_info_gain=0.4810644431995964), 9633: TreeNode(parent=card1, parent_branch=9633, attribute=C1, node_type=decision, class_val

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Value:9633 Children:{14649: TreeNode(parent=card1, parent_branch=14649, attribute=C6, node_type=decision, class_value=None, split=1.3, c_val=52.03268717699957, chi_val=3.841458820694124, max_info_gain=0.4011647793278923), 5376: TreeNode(parent=card1, parent_branch=5376, attribute=None, node_type=leaf, class_value=1, split=None, c_val=None, chi_val=None, max_info_gain=None), 6328: TreeNode(parent=card1, parent_branch=6328, attribute=None, node_type=leaf, class_value=0, split=None, c_val=None, chi_val=None, max_info_gain=None), 18084: TreeNode(parent=card1, parent_branch=18084, attribute=None, node_type=leaf, class_value=0, split=None, c_val=4.0, chi_val=5.991464547107979, max_info_gain=None), 9485: TreeNode(parent=card1, parent_branch=9485, attribute=ProductCD, node_type=decision, class_value=None, split=None, c_val=27.42857142857143, chi_val=7.814727903251179, max_info_gain=0.4810644431995964), 9633: TreeNode(parent=card1, parent_branch=9633, attribute=C1, node_type=decision, class_val

KeyboardInterrupt: 

In [ ]:
#mitchell_csv = pd.read_csv("drive/MyDrive/RandomForest/mitchell.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
mitchell_csv = pd.read_csv("drive/MyDrive/RandomForest/mitchell_missingvalues_num.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
mitchell_data = mitchell_csv.sample(frac=1, random_state=10).reset_index(drop=True) # shuffles data
#mitchell_csv_test = pd.read_csv("drive/MyDrive/RandomForest/mitchell_missingvalues_num_test.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
#mitchell_data_test = mitchell_csv_test.sample(frac=1, random_state=10).reset_index(drop=True) # shuffles data
#mitchell_data.replace("NoValue", np.nan, inplace=True)
#mitchell_data.replace("NotFound", np.nan, inplace=True)
#mitchell_data.replace("NaN", np.nan, inplace=True)
#mitchell_data.fillna(mitchell_data.mode().iloc[0])

data_types = {
    'Outlook': 'categorical',
    'Temperature': 'numerical',
    'Humidity': 'categorical',
    'Wind': 'categorical'
}

'''for column in mitchell_data.columns[1:-1]:
  mitchell_data[column].fillna(mitchell_data[column].mode().iloc[0], inplace = True)
print(mitchell_data)'''
test_tree = DecisionTree(mitchell_data, "PlayTennis", data_types, max_depth = 20, impurity_measure="entropy", confidence = 0.40)
#print(test_tree.dataset["PlayTennis"].mode().iloc[0])
#print(test_tree.calculate_entropy(mitchell_data))
#print(test_tree.calculate_gini(mitchell_data))
#print(test_tree.calculate_misclassification_error(mitchell_data))
#print("Outlook", test_tree.calculate_information_gain(mitchell_data, "Outlook"))
#print("Temperature", test_tree.calculate_information_gain(mitchell_data, "Temperature"))
#print("Humidity", test_tree.calculate_information_gain(mitchell_data, "Humidity"))
#print("Wind", test_tree.calculate_information_gain(mitchell_data, "Wind"))
#print("Outlook", test_tree.chi_square_test(mitchell_data, "Outlook"))
#print(test_tree.chi_square_test(mitchell_data, "Wind"))
#test_tree.impute_data(mitchell_data, "median")
#print(mitchell_data)
test_tree.train()
print("=====================")
#dfs(test_tree.tree)
#print(mitchell_data)
predictions = test_tree.predict(mitchell_data)
#print(mitchell_data["PlayTennis"])

predictions_results = [dictionary["Class"] for dictionary in predictions if "Class" in dictionary]
print(calculate_accuracy(mitchell_data["PlayTennis"], predictions_results) * 100)

In [ ]:
#current test here


mitchell_csv = pd.read_csv("drive/MyDrive/RandomForest/mitchell.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
mitchell_data = mitchell_csv.sample(frac=1, random_state=10).reset_index(drop=True) # shuffles data
#mitchell_csv_test = pd.read_csv("drive/MyDrive/RandomForest/mitchelltest.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
#mitchell_data_test = mitchell_csv_test.sample(frac=1, random_state=10).reset_index(drop=True) # shuffles data
#mitchell_data.replace("NoValue", np.nan, inplace=True)
#mitchell_data.replace("NotFound", np.nan, inplace=True)
#mitchell_data.replace("NaN", np.nan, inplace=True)
#mitchell_data.fillna(mitchell_data.mode().iloc[0])
#print(mitchell_data)

data_types = {
    'Outlook': 'categorical',
    'Temperature': 'categorical',
    'Humidity': 'categorical',
    'Wind': 'categorical'
}

'''for column in mitchell_data.columns[1:-1]:
  mitchell_data[column].fillna(mitchell_data[column].mode().iloc[0], inplace = True)
print(mitchell_data)'''
test_tree = DecisionTree(mitchell_data, "PlayTennis", data_types, max_depth = 120, impurity_measure="entropy",confidence = 0.20, id='Day')
#print(test_tree.dataset["PlayTennis"].mode().iloc[0])
#print(test_tree.calculate_entropy(mitchell_data))
#print(test_tree.calculate_gini(mitchell_data))
#print(test_tree.calculate_misclassification_error(mitchell_data))
print("Outlook", test_tree.calculate_information_gain(mitchell_data, "Outlook"))
print("Temperature", test_tree.calculate_information_gain(mitchell_data, "Temperature"))
print("Humidity", test_tree.calculate_information_gain(mitchell_data, "Humidity"))
print("Wind", test_tree.calculate_information_gain(mitchell_data, "Wind"))
print("Outlook", test_tree.chi_square_test(mitchell_data, "Outlook"))
#print(test_tree.chi_square_test(mitchell_data, "Wind"))
#test_tree.impute_data(mitchell_data, "median")
#print(mitchell_data)
test_tree.train()
print("=====================")
dfs(test_tree.tree)
#print(mitchell_data)
test_tree.predict(mitchell_data)

Outlook (0.24674981977443933, None)
Temperature (0.02922256565895487, None)
Humidity (0.15183550136234159, None)
Wind (0.04812703040826949, None)
Outlook (False, 3.546666666666666, 0.4462871026284195)
    Day   Outlook Temperature Humidity    Wind PlayTennis
0    D4      Rain        Mild     High    Weak        Yes
1    D8     Sunny        Mild     High    Weak         No
2   D13  Overcast         Hot   Normal    Weak        Yes
3    D7  Overcast        Cool   Normal  Strong        Yes
4    D9     Sunny        Cool   Normal    Weak        Yes
5    D3  Overcast         Hot     High    Weak        Yes
6   D11     Sunny        Mild   Normal  Strong        Yes
7    D6      Rain        Cool   Normal  Strong         No
8   D12  Overcast        Mild     High  Strong        Yes
9   D14      Rain        Mild     High  Strong         No
10   D2     Sunny         Hot     High  Strong         No
11   D1     Sunny         Hot     High    Weak         No
12   D5      Rain        Cool   Normal    Wea

0      {'Day': 'D4', 'Class': 'Yes'}
1       {'Day': 'D8', 'Class': 'No'}
2     {'Day': 'D13', 'Class': 'Yes'}
3      {'Day': 'D7', 'Class': 'Yes'}
4      {'Day': 'D9', 'Class': 'Yes'}
5      {'Day': 'D3', 'Class': 'Yes'}
6     {'Day': 'D11', 'Class': 'Yes'}
7       {'Day': 'D6', 'Class': 'No'}
8     {'Day': 'D12', 'Class': 'Yes'}
9      {'Day': 'D14', 'Class': 'No'}
10      {'Day': 'D2', 'Class': 'No'}
11      {'Day': 'D1', 'Class': 'No'}
12     {'Day': 'D5', 'Class': 'Yes'}
13    {'Day': 'D10', 'Class': 'Yes'}
dtype: object

In [ ]:
#current test here 222


mitchell_csv = pd.read_csv("drive/MyDrive/RandomForest/mitchell2.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
mitchell_data = mitchell_csv.sample(frac=1, random_state=10).reset_index(drop=True) # shuffles data
#mitchell_csv_test = pd.read_csv("drive/MyDrive/RandomForest/mitchelltest.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
#mitchell_data_test = mitchell_csv_test.sample(frac=1, random_state=10).reset_index(drop=True) # shuffles data
#mitchell_data.replace("NoValue", np.nan, inplace=True)
#mitchell_data.replace("NotFound", np.nan, inplace=True)
#mitchell_data.replace("NaN", np.nan, inplace=True)
#mitchell_data.fillna(mitchell_data.mode().iloc[0])
#print(mitchell_data)

data_types = {
    'Outlook': 'categorical',
    'Temperature': 'numerical',
    'Humidity': 'categorical',
    'Wind': 'categorical'
}

'''for column in mitchell_data.columns[1:-1]:
  mitchell_data[column].fillna(mitchell_data[column].mode().iloc[0], inplace = True)
print(mitchell_data)'''
test_tree = DecisionTree(mitchell_data, "PlayTennis", data_types, max_depth = 120, impurity_measure="entropy",confidence = 0.95, id='Day')
#print(test_tree.dataset["PlayTennis"].mode().iloc[0])
#print(test_tree.calculate_entropy(mitchell_data))
#print(test_tree.calculate_gini(mitchell_data))
#print(test_tree.calculate_misclassification_error(mitchell_data))
print("Outlook", test_tree.calculate_information_gain(mitchell_data, "Outlook"))
print("Temperature", test_tree.calculate_information_gain(mitchell_data, "Temperature"))
print("Humidity", test_tree.calculate_information_gain(mitchell_data, "Humidity"))
print("Wind", test_tree.calculate_information_gain(mitchell_data, "Wind"))
print("Outlook", test_tree.chi_square_test(mitchell_data, "Outlook"))


bsplit = test_tree.find_best_split(mitchell_data, 'Temperature')
print(bsplit)
cval = test_tree.chi_square_test(mitchell_data, 'Temperature', bsplit)
print(cval)
chi_square_value = chi2.ppf(0.95, 1)
print(chi_square_value)

#print(test_tree.chi_square_test(mitchell_data, "Wind"))
#test_tree.impute_data(mitchell_data, "median")
#print(mitchell_data)
test_tree.train()
print("=====================")
dfs(test_tree.tree)
#print(mitchell_data)
test_tree.predict(mitchell_data)



Outlook (0.24674981977443933, None)
Temperature (0.9402859586706311, None)
Humidity (0.15183550136234159, None)
Wind (0.04812703040826949, None)
Outlook (True, 3.546666666666666, 5.991464547107979)
60.0
(True, 1.2962962962962963, 3.841458820694124)
3.841458820694124
    Day   Outlook  Temperature Humidity    Wind PlayTennis
0    D4      Rain           70     High    Weak        Yes
1    D8     Sunny           68     High    Weak         No
2   D13  Overcast           93   Normal    Weak        Yes
3    D7  Overcast           62   Normal  Strong        Yes
4    D9     Sunny           60   Normal    Weak        Yes
5    D3  Overcast           88     High    Weak        Yes
6   D11     Sunny           74   Normal  Strong        Yes
7    D6      Rain           61   Normal  Strong         No
8   D12  Overcast           70     High  Strong        Yes
9   D14      Rain           71     High  Strong         No
10   D2     Sunny           92     High  Strong         No
11   D1     Sunny        

0      {'Day': 'D4', 'Class': 'Yes'}
1      {'Day': 'D8', 'Class': 'Yes'}
2     {'Day': 'D13', 'Class': 'Yes'}
3      {'Day': 'D7', 'Class': 'Yes'}
4      {'Day': 'D9', 'Class': 'Yes'}
5      {'Day': 'D3', 'Class': 'Yes'}
6     {'Day': 'D11', 'Class': 'Yes'}
7      {'Day': 'D6', 'Class': 'Yes'}
8     {'Day': 'D12', 'Class': 'Yes'}
9     {'Day': 'D14', 'Class': 'Yes'}
10     {'Day': 'D2', 'Class': 'Yes'}
11     {'Day': 'D1', 'Class': 'Yes'}
12     {'Day': 'D5', 'Class': 'Yes'}
13    {'Day': 'D10', 'Class': 'Yes'}
dtype: object

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming df is your DataFrame and 'categorical_column' is the column with categorical values
label_encoder = LabelEncoder()
mitchell_data['Outlook'] = label_encoder.fit_transform(mitchell_data['Outlook'])

print(mitchell_data)




NameError: name 'mitchell_data' is not defined

In [ ]:
mitchell_csv = pd.read_csv("drive/MyDrive/RandomForest/mitchell_missingvalues.csv", na_values=['NaN', 'NotFound', 'nan', 'NoValue'])
mitchell_data = mitchell_csv.sample(frac=1, random_state=10).reset_index(drop=True) # shuffles data
#mitchell_data.replace("NoValue", np.nan, inplace=True)
#mitchell_data.replace("NotFound", np.nan, inplace=True)
#mitchell_data.replace("NaN", np.nan, inplace=True)
#mitchell_data.fillna(mitchell_data.mode().iloc[0])

data_types = {
    'Outlook': 'categorical',
    'Temperature': 'categorical',
    'Humidity': 'categorical',
    'Wind': 'categorical'
}

'''for column in mitchell_data.columns[1:-1]:
  mitchell_data[column].fillna(mitchell_data[column].mode().iloc[0], inplace = True)
print(mitchell_data)'''
test_tree = DecisionTree(mitchell_data, "PlayTennis", data_types, max_depth = 5, impurity_measure="entropy",confidence = 0.8)
#print(test_tree.dataset["PlayTennis"].mode().iloc[0])
#print(test_tree.calculate_entropy(mitchell_data))
#print(test_tree.calculate_gini(mitchell_data))
#print(test_tree.calculate_misclassification_error(mitchell_data))
print("Outlook", test_tree.calculate_information_gain(mitchell_data, "Outlook"))
print("Temperature", test_tree.calculate_information_gain(mitchell_data, "Temperature"))
print("Humidity", test_tree.calculate_information_gain(mitchell_data, "Humidity"))
print("Wind", test_tree.calculate_information_gain(mitchell_data, "Wind"))
print("Outlook", test_tree.chi_square_test(mitchell_data, "Outlook"))
#print(test_tree.chi_square_test(mitchell_data, "Wind"))
#test_tree.impute_data(mitchell_data, "median")
#print(mitchell_data)
test_tree.train()
print("=====================")
dfs(test_tree.tree)
#print(mitchell_data)
test_tree.predict(mitchell_data)

Outlook 0.3734953562613922
Temperature 0.18385092540042125
Humidity 0.43454913999520245
Wind 0.052764999427602044
Outlook (False, 4.8621428571428575, 4.64162767608745)
    Day   Outlook Temperature Humidity    Wind PlayTennis
0    D4      Rain        Mild     High    Weak        Yes
1    D8     Sunny        Mild     High    Weak         No
2   D13  Overcast         Hot   Normal    Weak        Yes
3    D7  Overcast        Cool   Normal  Strong        Yes
4    D9     Sunny        Cool   Normal    Weak        Yes
5    D3  Overcast         Hot     High    Weak        Yes
6   D11      Rain        Mild   Normal  Strong        Yes
7    D6      Rain        Cool   Normal    Weak        Yes
8   D12  Overcast        Mild     High  Strong        Yes
9   D14      Rain        Mild     High  Strong         No
10   D2     Sunny         Hot     High  Strong         No
11   D1     Sunny         Hot     High    Weak         No
12   D5      Rain        Cool     High    Weak        Yes
13  D10      Rain   

0      {'Day': 'D4', 'Class': 'Yes'}
1       {'Day': 'D8', 'Class': 'No'}
2     {'Day': 'D13', 'Class': 'Yes'}
3      {'Day': 'D7', 'Class': 'Yes'}
4       {'Day': 'D9', 'Class': 'No'}
5      {'Day': 'D3', 'Class': 'Yes'}
6      {'Day': 'D11', 'Class': 'No'}
7      {'Day': 'D6', 'Class': 'Yes'}
8     {'Day': 'D12', 'Class': 'Yes'}
9      {'Day': 'D14', 'Class': 'No'}
10      {'Day': 'D2', 'Class': 'No'}
11      {'Day': 'D1', 'Class': 'No'}
12     {'Day': 'D5', 'Class': 'Yes'}
13    {'Day': 'D10', 'Class': 'Yes'}
dtype: object

In [ ]:
tree = DecisionTree(training_data, "isFraud", feature_type)
#print(tree.calculate_gini(training_data, "ProductCD"))
#print(tree.calculate_entropy(training_data, "ProductCD"))

#print(tree.calculate_entropy(training_data))
#print(tree.calculate_misclassification_error(training_data, "ProductCD"))
#print(tree.information_gain(training_data, "ProductCD", "gini"))

# print(tree.find_best_split(training_data, 'TransactionAmt'))

# print(training_data["isFraud"].value_counts())
#values = training_data["isFraud"].value_counts()
#for value in values:
#  print(value)
print(training_data["isFraud"].value_counts()[1])

0.21897805432701886
13236


Class containing Random Tree information

In [ ]:
class RandomTree:
  pass

In [ ]:
def gini(c):

  if isinstance(c, pd.Series):
    p = c.value_counts()/c.shape[0]
    gini = 1-np.sum(p**2)
    return(gini)


print(gini(training_data.isFraud))


In [ ]:
def entropy(c):

  if isinstance(c, pd.Series):

    a = c.value_counts()/c.shape[0]
    print(a)
    #entropy = np.sum(-a*np.log2(a+1e-9))
    entropy = np.sum(-(a*np.log2(a+1e-9)))
    return(entropy)

print(entropy(training_data.isFraud))


In [ ]:
def misclass(c):

  if isinstance(c, pd.Series):
    p = c.value_counts()/c.shape[0]
    e = 1-max(p)
    return(e)


print(misclass(training_data.isFraud))


In [ ]:
def calculate_impurity(data, impurity_measure):
  if impurity_measure == "gini":
    return gini(data)
  elif impurity_measure == "entropy":
    return entropy(data)
  elif impurity_measure == "misclassification_error":
    return misclass(data)

def information_gain(data, attribute, impurity_measure = "gini"):
    impurity = calculate_impurity(data[attribute], impurity_measure)
    info_gain = impurity - (data.groupby(attribute).apply(lambda x : len(x) / len(data) * calculate_impurity(x[attribute], impurity_measure)).sum())
    return info_gain

print(information_gain(training_data, "isFraud"))